Допустим, у нас есть $n$ товаров с заданными стоимостями $v_i$ и массой $w_i$. В сумку убирается $С$ кг. Сколько какого товара взять, чтобы сумма всех стоимостей товаров была наибольшей?

In [ ]:
values = [4, 2, 1, 7, 3, 6]
weights = [5, 9, 8, 2, 6, 5]
C = 15
n = 6

Сформулируем задачу:
$$\max\sum v_i x_i$$
$$\sum w_i x_i \leq C $$

Как должна выглядеть задача:
$$\min c^T x$$
$$A x \leq b $$

Получается, что $c=-v$, $A=w^T$, $b=(C)$

In [ ]:
import numpy as np

In [ ]:
c = - np.array(values)
A = np.array(weights)         #shape = (6,)
A = np.expand_dims(A, 0)      #shape = (1,6)
b = np.array([C])

In [ ]:
b

array([15])

In [ ]:
from scipy.optimize import linprog

In [ ]:
linprog(c=c, A_ub=A, b_ub=b)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


     con: array([], dtype=float64)
     fun: -52.50000000003075
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([-2.24904539e-11])
  status: 0
 success: True
       x: array([6.18738527e-14, 1.05853305e-12, 1.21475942e-13, 7.50000000e+00,
       4.00246688e-13, 4.71394158e-13])

In [ ]:
!pip install cvxpy

In [ ]:
import cvxpy

In [ ]:
x = cvxpy.Variable(shape=n, integer = True)

In [ ]:
constraint = (A @ x <= b)
total_value = c * x

In [ ]:
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint])

In [ ]:
problem.solve()

-inf

In [ ]:
x.value

Теперь положительные $x$

In [ ]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = (A @ x <= b)
x_positive = (x >= 0)
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

-49.0

In [ ]:
x.value

array([0., 0., 0., 7., 0., 0.])

Теперь $x = 0$ или $1$

In [ ]:
# A матрица весов
# c -1 * вектор стоимости
# x переменная в форме n  -- c * x - объективная функция
# b ограничение по весу

array([15])

In [ ]:
c

array([-4, -2, -1, -7, -3, -6])

In [ ]:
x = cvxpy.Variable(shape=n, boolean=True)
constraint = A @ x <= b
x_positive = x >= 0
total_value = c * x
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=[constraint, x_positive])
problem.solve()

-17.0

In [ ]:
x.value

array([1., 0., 0., 1., 0., 1.])

Задание 5.1

Составьте оптимальный план перевозок, со Склада № 1 и Склада № 2, в три торговых центра, с учётом тарифов, запасов и потребностей, которые указаны в таблице:

--- | ТЦ1 (110 шт.) | ТЦ2 (250) | ТЦ3 (140)
---- |---| --- | ----
склад 1 (180) | 2 уе | 2 | 3
склад 2 (220) | 7 | 7 | 6


Сформулируйте задачу, как задачу линейного программирования.

В ответ запишите минимальную суммарную стоимость поставки (с точностью до целых)

In [ ]:
# http://yetanothermathprogrammingconsultant.blogspot.com/2019/11/cvxpy-matrix-style-modeling-limits.html
import numpy as np
import cvxpy as cp

#----- data -------
capacity = np.array([180, 220])
demand = np.array([110, 150, 140])
cost = np.array([[2, 5, 3],
            [7, 7, 6]])
# freight = 90
# cost = freight*distance/1000

#------ set up LP data --------
C = cost
d = demand
s = capacity

#---- matrix formulation ----

ei = np.ones(s.shape)
ej = np.ones(d.shape)

X = cp.Variable(C.shape,"X")
prob = cp.Problem(
    cp.Minimize(cp.trace(C.T@X)),
    [X.T@ei >= d, 
     X@ej <= s, 
     X>=0])
prob.solve(verbose=True)
print("status:",prob.status)
print("objective:",prob.value)
print("levels:",X.value)

#---- summations ----

prob2 = cp.Problem(
    cp.Minimize(cp.sum(cp.multiply(C,X))),
    [cp.sum(X,axis=0) >= d,
    cp.sum(X,axis=1) <= s,
    X >= 0])
prob2.solve(verbose=True)
print("status:",prob2.status)
print("objective:",prob2.value)
print("levels:",X.value)


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +1.733e+03  +1.733e+03  +3e+03  1e-01  4e-01  1e+00  3e+02    ---    ---    1  1  - |  -  - 
 1  +1.994e+03  +1.999e+03  +4e+02  1e-02  4e-02  5e+00  3e+01  0.8994  2e-02   0  0  0 |  0  0
 2  +1.909e+03  +1.911e+03  +7e+01  2e-03  8e-03  2e+00  6e+00  0.8708  6e-02   0  0  0 |  0  0
 3  +1.900e+03  +1.900e+03  +2e+00  7e-05  3e-04  8e-02  2e-01  0.9690  5e-03   0  0  0 |  0  0
 4  +1.900e+03  +1.900e+03  +3e-02  8e-07  3e-06  9e-04  2e-03  0.9890  1e-04   1  0  0 |  0  0
 5  +1.900e+03  +1.900e+03  +3e-04  9e-09  3e-08  1e-05  3e-05  0.9890  1e-04   1  0  0 |  0  0
 6  +1.900e+03  +1.900e+03  +3e-06  1e-10  4e-10  1e-07  3e-07  0.9890  1e-04   2  0  0 |  0  0

OPTIMAL (within feastol=3.7e-10, reltol=1.7e-09, abstol=3.3e-06).
Runtime: 0.001747 seconds.

status: optimal
objective: 1900.0000000153718
l

In [ ]:
import numpy as np
import cvxpy

# Инициализация матрицы стоимости и матрицы, которая будет задавать поставки:
c = np.array([[2, 5, 3], [7, 7, 6]])
x = cvxpy.Variable(shape=(2, 3), integer=True)

# Ограничения для задачи: у нас есть ограничения на объемы продукции в складах 
# - это даёт ограничения-неравенства на суммы по рядам, 
# и ограничения, связанные с требованиями ТЦ 
# - это даёт ограничения-равенства на суммы по столбцам:

constraint = [cvxpy.sum(x[0]) <= 180, 
              cvxpy.sum(x[1]) <= 220, 
              cvxpy.sum(x[:, 0]) == 110, 
              cvxpy.sum(x[:, 1]) == 150, 
              cvxpy.sum(x[:, 2]) == 140, 
              x >= 0]

# Непосредственно минимизируемая функция 
# - сумма поэлементных произведений матриц x и c :        

total_value = cvxpy.sum(cvxpy.multiply(c, x))
problem = cvxpy.Problem(cvxpy.Minimize(total_value), constraints=constraint)
problem.solve(solver='ECOS_BB')

1900.0000000102355

In [ ]:
x.value

array([[1.10000000e+02, 3.50528004e-09, 7.00000000e+01],
       [3.36650313e-09, 1.50000000e+02, 7.00000000e+01]])

In [ ]:
from scipy.optimize import linprog
import numpy as np

c = np.array([2, 5, 3, 7, 7, 6])

A_ub = np.array([[1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 1, 1]])
b_ub = np.array([180, 220])

A_eq = np.array([[1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1]])
b_eq = np.array([110, 150, 140])

args = linprog(c=c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq)
args

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


     con: array([4.39231928e-06, 6.01910409e-06, 5.61240802e-06])
     fun: 1899.9999256826418
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([7.20365003e-06, 8.82018139e-06])
  status: 0
 success: True
       x: array([1.09999995e+02, 4.43103329e-08, 6.99999980e+01, 8.98577274e-07,
       1.49999994e+02, 6.99999963e+01])

Задание 5.2

Решите задачу о назначениях

колонки - задачи, строки - исполнители

В ответ запишите минимальную стоимость

 -- | 1 | 2 | 3 | 4 | 5 
 -- | -| - |  -| -| -
 1 | 1000 | 12 | 10 | 19 | 8
 2 | 12 | 1000 | 3 | 7| 2
 3 | 10 | 3 | 1000 | 6 |20
 4 | 19 | 7 |6 |1000 |4
 5 |8 |2 |20 | 4 |1000


In [2]:
# Задание 5.2
import cvxpy as cvx
import numpy as np

c = np.array([[1000, 12,   10,   19,   8],
              [12,   1000, 3,    7,    2],
              [10,   3,    1000, 6,    20],
              [19,   7,    6,    1000, 4],
              [8,    2,    20,   4,    1000]])

# описывает параметры переменной х пока без задания мест расположения 0 и 1
x = cvx.Variable(shape=(5,5), boolean=True)

#это суммы по столбцам и строкам, которые должны быть равны единицам. 
# Указаны как равенство векторов сумм по столбцам и строкам векторам, состоящих из единиц
constraints = [cvx.sum(x, axis=0) == np.ones(5), cvx.sum(x, axis=1) == np.ones(5)]

func = cvx.sum(cvx.multiply(x, c))

#problem выбирает такое расположение  0 и 1  в х,  чтобы минимизировать func.
problem = cvx.Problem(cvx.Minimize(func), constraints=constraints)
problem.solve(solver='ECOS_BB')

31.999999999961364

In [3]:
# Задание 5.3

# Необходимо найти кратчайший маршрут из точки А, который проходит через все другие 
# точки и возвращается в А.
# Сформулируйте эту задачу как задачу ЦЛП и решите её.
# В ответ запишите длину кратчайшего пути

import cvxpy
import numpy as np
x = cvxpy.Variable(shape=(5,5), boolean=True)
u = cvxpy.Variable(shape=5, integer=True)

from itertools import product

# матрица стоимостей переходов (длина маршрута) составляется для вершин в алфавитном порядке:
# первая строка/столбец - это вершина A, вторая строка/столбец - вершина B и т.д.

# элемент d[0, 1] - это цена перехода из вершины A в вершину B, и так далее.

constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5),
    cvxpy.sum(x, axis=1) == np.ones(5),
    u >= 0,
    u <= 4,
    cvxpy.sum(cvxpy.diag(x)) == 0
]

# Это условия, которые должны обеспечить связность графа, 
# то есть то что маршрут не будет разбит на независимые подциклы
for i, j in product(range(5), range(5)):
    if i >= 0 and j >= 1 and i != j:
        constraints.append(u[i] - u[j] + 5 * x[i,j] <= 4)

d = np.array([[0, 12, 10, 19, 8], 
              [12, 0, 3, 7, 2], 
              [10, 3, 0, 6, 20], 
              [19, 7, 6, 0, 4], 
              [8, 2, 20, 4, 0]])

func = cvxpy.sum(cvxpy.multiply(x, d))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
result = problem.solve(solver='ECOS_BB')
np.round(result)

32.0

In [4]:
# альтернативное решение

# адание 5.3 может быть решено и без условий, добавляемых в циклеб
# потому что в этом графе  такой ситуации возникнуть не может, 
# а для 6 точек уже возможны несвязные подмаршруты, поэтому условия связности понадобятся

import cvxpy
import numpy as np

x = cvxpy.Variable(shape=(5,5), boolean=True)

# матрица стоимостей переходов из вершины в вершину
d = np.array([
    # A   B   C   D   E
    [ 0, 12, 10, 19,  8], # A
    [12,  0,  3,  7,  2], # B
    [10,  3,  0,  6, 20], # C
    [19,  7,  6,  0,  4], # D
    [ 8,  2, 20,  4,  0]])# E

constraints = [
    cvxpy.sum(x, axis=0) == np.ones(5), # в каждой строке может быть выбран только один переход
    cvxpy.sum(x, axis=1) == np.ones(5), # в каждом столбце может быть выбран только один переход
    cvxpy.sum(cvxpy.diag(x)) == 0,      # не может быть выбран переход из точки в ту же точку
]

func = cvxpy.sum(cvxpy.multiply(x, d))
problem = cvxpy.Problem(cvxpy.Minimize(func), constraints=constraints)
result = problem.solve(solver='ECOS_BB')
np.round(result)

32.0